In [1]:
import os
from dotenv import load_dotenv


load_dotenv()  # .envファイルから環境変数を読み込み


True

In [2]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.googlesearch import GoogleSearchTools  # 正しいクラス名

# シンプルなエージェントの作成
agent = Agent(
    model=OpenAIChat(id="gpt-4o"),  # gpt-4o を使う
    tools=[GoogleSearchTools()],  # GoogleSearchToolsを使用
    markdown=True
)

# エージェントに質問を投げかける
response = agent.print_response("今日の東京の天気は？", stream=True)
print(response)

Output()

None


In [3]:
# より複雑な例：知識ベースを持つエージェント
from agno.embedder.openai import OpenAIEmbedder
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.lancedb import LanceDb, SearchType

# 知識ベース付きエージェントの作成
agent_with_knowledge = Agent(
    model=OpenAIChat(id="gpt-4o"),  # gpt-4o を使う
    description="財務諸表の分析エキスパートである経営者",
    instructions=[
        "知識ベースから財務諸表を検索します。",
        "必要に応じてGoogle検索で補完します。",
        "知識ベースの情報を優先して使用します。"
    ],
    knowledge=PDFUrlKnowledgeBase(
        urls=["https://www.fastretailing.com/jp/ir/library/pdf/tanshin202408_4q.pdf"],  # あなたのPDFのURLに置き換えてください
        vector_db=LanceDb(
            uri="data/lancedb",
            table_name="finance",
            search_type=SearchType.hybrid,
            embedder=OpenAIEmbedder(id="text-embedding-3-small"),
        ),
    ),
    tools=[GoogleSearchTools()],
    show_tool_calls=True,
    markdown=True
)

# 知識ベースのロード（初回のみ必要）
if agent_with_knowledge.knowledge is not None:
    agent_with_knowledge.knowledge.load()

# エージェントへの質問例
response = agent_with_knowledge.print_response("ファーストリテイリングの財務諸表を分析して", stream=True)
print(response)


INFO     Loading knowledge base

INFO     Reading: https://www.fastretailing.com/jp/ir/library/pdf/tanshin202408_4q.pdf

INFO     No documents to insert

INFO     Added 0 documents to knowledge base

Output()

None
